In [ ]:
from finn.util.visualization import showInNetron
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
import json

build_dir = "/workspace/samo/outputs/saved/finn"

with open("config.json", "r") as f:
    model_config = json.load(f)

model_name = model_config["model_name"]

model = ModelWrapper(build_dir+"/{}_post_optimiser.onnx".format(model_name))
showInNetron(build_dir+"/{}_post_optimiser.onnx".format(model_name))

## 3. Hardware Build <a id='vivado'></a>

We're finally ready to start generating hardware from our network. Depending on whether you want to target a Zynq or Alveo platform, FINN offers two transformations to build the accelerator, integrate into an appropriate shell and build a bitfile. These are `ZynqBuild` and `VitisBuild` for Zynq and Alveo, respectively. In this notebook we'll demonstrate the `ZynqBuild` as these boards are more common and it's much faster to complete bitfile generation for the smaller FPGAs found on them.

As we will be dealing with FPGA synthesis tools in these tasks, we'll define two helper variables that describe the Xilinx FPGA part name and the PYNQ board name that we are targeting.

In [ ]:
from finn.util.basic import pynq_part_map, alveo_part_map, alveo_default_platform
pynq_part_map["Zedboard"] = "xc7z020clg484-1"
pynq_part_map["ZC706"] = "xc7z045ffg900-2"

device_name = model_config["device"]
target_clk_ns = model_config["clock_cycle"]

In previous versions of FINN, we had to manually go through several steps to generate HLS code, stitch IP, create a PYNQ project and run synthesis. All these steps are now performed by the `ZynqBuild` transform (or the `VitisBuild` transform for Alveo). **As this involves calling HLS synthesis and Vivado synthesis, this transformation will run for some time (up to half an hour depending on your PC).**

In [ ]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.transformation.fpgadataflow.vitis_build import VitisBuild, VitisOptStrategy

if device_name in pynq_part_map.keys():
    model = model.transform(ZynqBuild(platform = device_name, period_ns = target_clk_ns))
elif device_name in alveo_part_map.keys():
    model = model.transform(VitisBuild(fpga_part=alveo_part_map[device_name], period_ns = target_clk_ns, platform = alveo_default_platform[device_name], strategy=VitisOptStrategy.BUILD_SPEED))
else:
    assert False

In [ ]:
model.save(build_dir + "/{}_post_synthesis.onnx".format(model_name))

### Examining the generated outputs <a id='gen_outputs'></a>

Let's start by viewing the post-synthesis model in Netron:

### Model Prediction

In [ ]:
model = ModelWrapper(build_dir+"/{}_post_optimiser.onnx".format(model_name))
showInNetron(build_dir+"/{}_post_optimiser.onnx".format(model_name))

In [ ]:
from finn.transformation.fpgadataflow.annotate_resources import AnnotateResources
model = model.transform(AnnotateResources(mode="estimate"))
estimate_network_resource = model.get_metadata_prop("res_total_estimate")

from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
# need to call AnnotateCycles before dataflow_performance
model = model.transform(AnnotateCycles())
estimate_network_performance = model.analysis(dataflow_performance)

with open("report.txt", "w") as f:
    print("Model Prediction:", file=f)
    print(estimate_network_resource, file=f)
    print(estimate_network_performance, file=f)

### HLS Synthesis

In [ ]:
model = ModelWrapper(build_dir + "/{}_post_synthesis.onnx".format(model_name))
showInNetron(build_dir+"/{}_post_synthesis.onnx".format(model_name))

In [ ]:
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")
model = ModelWrapper(postsynth_layers)
model = model.transform(AnnotateResources(mode="hls"))
hls_network_resource = model.get_metadata_prop("res_total_hls")

with open("report.txt", "a") as f:
    print("HLS Synthesis:", file=f)
    print(hls_network_resource, file=f)

### Place and Route

In [ ]:
#model = ModelWrapper(build_dir + "/{}_post_synthesis.onnx".format(model_name))
#showInNetron(build_dir+"/{}_post_synthesis.onnx".format(model_name))

In [ ]:
#model = model.transform(AnnotateResources(mode="synth"))
#synth_network_resource = model.get_metadata_prop("res_total_top_synth")
#with open("report.txt", "a") as f:
#    print("Place and Route:", file=f)
#    print(synth_network_resource, file=f)